<a href="https://colab.research.google.com/github/timroelofs123/face_reaging/blob/main/notebooks/gradio_demos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Re-Aging Demos with Gradio; From the face_reaging repository. https://github.com/timroelofs123/face_reaging/blob/main/notebooks/gradio_demos.ipynb
To try out the Gradio Demos using Google Colab, run the cells below.
Be sure to select a runtime with GPU (e.g. `Runtime > Change Runtime Type > T4 (GPU)`) to make the model run fast.

You can choose to either run the regular inference demo or the video demo. The demo will open in a new tab.

## Downloading files and model

In [1]:
!git clone https://github.com/timroelofs123/face_reaging

fatal: destination path 'face_reaging' already exists and is not an empty directory.


In [2]:
!git clone https://huggingface.co/timroelofs123/face_re-aging

fatal: destination path 'face_re-aging' already exists and is not an empty directory.


In [3]:
%mv face_re-aging/best_unet_model.pth face_reaging/
%cd face_reaging/

mv: rename face_re-aging/best_unet_model.pth to face_reaging/best_unet_model.pth: No such file or directory
/Users/takumifujiwara/BrownAcademics/2024FallSemester/CSCI1430ComputerVision/FinalProject/CS1430/face_reaging


## Installing requirements

In [4]:
%pip install gradio face_recognition antialiased_cnns kaleido av --quiet

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      running bdist_wheel
      running build
      running build_ext
      
      
                         CMake is not installed on your system!
      
          Or it is possible some broken copy of cmake is installed on your system.
          It is unfortunately very common for python package managers to include
          broken copies of cmake.  So if the error above this refers to some file
          path to a cmake file inside a python or anaconda or miniconda path then you
          should delete that broken copy of cmake from your computer.
      
          Instead, please get an official copy of cmake from one of these known good
          sources of an official cmake:
              - cmake.org (this is how windows users should get cmake)
              - apt install cmake (for Ubuntu or Debian based systems)
              - yum instal

## Running demo

Re-age your photo or video with the three demos.

In [6]:
import gradio as gr
import torch
from model.models import UNet
from scripts.test_functions import process_image, process_video

ModuleNotFoundError: No module named 'gradio'

In [11]:
# default settings
window_size = 512
stride = 256
steps = 18
frame_count = 100
model_path = 'best_unet_model.pth'

In [14]:
# The code from this block equivalent to the "run" function in scripts.gradio_demo

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(model_path, map_location=device))
unet_model.eval()

def block_img(image, source_age, target_age):
    return process_image(unet_model, image, video=False, source_age=source_age,
                          target_age=target_age, window_size=window_size, stride=stride)

def block_img_vid(image, source_age):
    return process_image(unet_model, image, video=True, source_age=source_age,
                          target_age=0, window_size=window_size, stride=stride, steps=steps)

def block_vid(video_path, source_age, target_age):
    return process_video(unet_model, video_path, source_age, target_age,
                          window_size=window_size, stride=stride, frame_count=frame_count)

demo_img = gr.Interface(
    fn=block_img,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(0, 90, value=20, step=1, label="Current age", info="Choose your current age"),
        gr.Slider(0, 90, value=80, step=1, label="Target age", info="Choose the age you want to become")
    ],
    outputs="image",
    examples=[
        ['UTK_processed/chosen_data/001_man_18_SA.jpg', 18, 50],
        ['UTK_processed/chosen_data/001_woman_18_white.jpg', 18, 50],
        ['UTK_processed/chosen_data/009_man_41_asian.jpg', 41, 80],
        ['UTK_processed/chosen_data/009_woman_53_black.jpg', 53, 10],
        ['UTK_processed/chosen_data/017_man_61_SA.jpg', 61, 15],
        ['UTK_processed/chosen_data/017_woman_76_indigenous?viet?.jpg', 76, 15]
    ],
    description="Input an image of a person and age them from the source age to the target age."
)

demo_img_vid = gr.Interface(
    fn=block_img_vid,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(0, 90, value=20, step=1, label="Current age", info="Choose your current age"),
    ],
    outputs=gr.Video(),
    examples=[
        ['UTK_processed/chosen_data/001_man_18_SA.jpg', 18],
        ['UTK_processed/chosen_data/001_woman_18_white.jpg', 18],
        ['UTK_processed/chosen_data/009_man_41_asian.jpg', 41],
        ['UTK_processed/chosen_data/009_woman_53_black.jpg', 53],
        ['UTK_processed/chosen_data/017_man_61_SA.jpg', 61],
        ['UTK_processed/chosen_data/017_woman_76_indigenous?viet?.jpg', 76]
    ],
    description="Input an image of a person and a video will be returned of the person at different ages."
)

demo_vid = gr.Interface(
    fn=block_vid,
    inputs=[
        gr.Video(),
        gr.Slider(10, 90, value=20, step=1, label="Current age", info="Choose your current age"),
        gr.Slider(10, 90, value=80, step=1, label="Target age", info="Choose the age you want to become")
    ],
    outputs=gr.Video(),
    examples=[
        ['UTK_processed/chosen_data/009_man_41_asian.jpg', 10, 80],
    ],
    description="Input a video of a person, and it will be aged frame-by-frame."
)

demo = gr.TabbedInterface([demo_img, demo_img_vid, demo_vid],
                          tab_names=['Image inference demo', 'Image animation demo', 'Video inference demo'],
                          title="Face Re-Aging Demo",
                          )

NameError: name 'torch' is not defined

In [ ]:
demo.launch()